In [10]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf
from transformers import BertTokenizer, BertTokenizerFast

pre_train_model = "bert-base-chinese"
tokenizer = BertTokenizerFast.from_pretrained(pre_train_model, do_lower_case=True)

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

In [2]:
import pandas as pd
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
import json
from pandas import json_normalize
from sklearn.model_selection import train_test_split

f = open("train.json", "r", encoding="utf-8")
info = json.load(f)
f.close()

data = []
column_names = ['id', 'utterance', 'emotion']
for k, v in info.items() :
   data.extend([ [key, i['utterance'], i['emotion']] for s in v for key, i in s.items() ])
        # print(key, i['utterance'], i['emotion'])
    # break
    
data_df = pd.DataFrame(data, columns=column_names)
data_df["emotion"] = le.fit_transform(data_df.emotion)
Xtrain, Xtest = train_test_split(data_df, test_size=0.25, random_state=42)

In [3]:
data_df

,id,utterance,emotion
0,1_1,那個憨女人有什麼值得送的，正鵬這個人也真是的！,1
1,1_2,哎喲，老婆子，你怎麼盡講那些不利於團結的話呢！他去送送他的同學也在情理之中嘛！,6
2,1_3,爸、媽，我回來啦！,4
3,1_4,我怕你喝了迷魂湯，魂被你那個憨同學勾引去了！,1
4,1_5,老婆子，看你講話像個母親說的嗎！你怎麼老是反對他們倆的事呢？,6
...,...,...,...
20609,4186_1,不光是看，我還要說。,3
20610,4186_2,說什麼？,6
20611,4186_3,說我愛你！我要說一百遍一千遍，我愛你！我愛你！我感覺到能在你面前這樣痛痛快快地說出來也是一種幸福。,3
20612,4187_1,我咋看不出你愛我？,3


In [4]:
# can be up to 512 for BERT
max_length = 49
batch_size = 20

def convert_example_to_feature(review):
  return tokenizer.encode_plus(review,
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = max_length, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
              )

def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(ds, limit=-1):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []

  if (limit > 0):
      ds = ds.take(limit)

  for index, row in ds.iterrows():
    bert_input = convert_example_to_feature(row.utterance)
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    # label_list.append(one_hot.loc[index].values)
    label_list.append([row.emotion])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)


# train dataset
ds_train_encoded = encode_examples(Xtrain).shuffle(10000).batch(batch_size)
# test dataset
ds_test_encoded = encode_examples(Xtest).batch(batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [11]:
# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-6
# we will do just 1 epoch, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 50
# model initialization
# model = TFBertForSequenceClassification.from_pretrained(pre_train_model, num_labels=32)
model = TFBertForSequenceClassification.from_pretrained(pre_train_model, num_labels=32)
# model = AutoModel.from_pretrained(pre_train_model, num_labels=32)

# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_test_encoded, callbacks=[callback])

Epoch 1/50
1546/1546 [==============================] - 145s 86ms/step - loss: 1.7858 - accuracy: 0.4504 - val_loss: 1.3581 - val_accuracy: 0.5444
Epoch 2/50
1546/1546 [==============================] - 129s 84ms/step - loss: 1.2613 - accuracy: 0.5682 - val_loss: 1.2526 - val_accuracy: 0.5617
Epoch 3/50
1546/1546 [==============================] - 129s 83ms/step - loss: 1.1081 - accuracy: 0.6142 - val_loss: 1.2191 - val_accuracy: 0.5704
Epoch 4/50
1546/1546 [==============================] - 126s 82ms/step - loss: 0.9925 - accuracy: 0.6553 - val_loss: 1.2131 - val_accuracy: 0.5759
Epoch 5/50
1546/1546 [==============================] - 125s 81ms/step - loss: 0.8888 - accuracy: 0.6909 - val_loss: 1.2423 - val_accuracy: 0.5797
Epoch 6/50
1546/1546 [==============================] - 126s 81ms/step - loss: 0.7874 - accuracy: 0.7289 - val_loss: 1.2956 - val_accuracy: 0.5704
Epoch 7/50
1546/1546 [==============================] - 126s 81ms/step - loss: 0.6921 - accuracy: 0.7633 - val_loss: 1

In [13]:
f = open("test.json", "r", encoding="utf-8")
info = json.load(f)
f.close()

data = []
column_names = ['id', 'utterance']
for k, v in info.items() :
    data.extend([ [key, i['utterance']] for s in v for key, i in s.items() ])
    # for s in v :
    #     for key, i in s.items():
    #         print(i)

data_df = pd.DataFrame(data, columns=column_names)
ans = []
for i in data_df.utterance :
    predict_input = tokenizer.encode(i, truncation=True,padding=True,return_tensors="tf")
    tf_output = model.predict(predict_input, verbose=0)[0]
    tf_prediction = tf.nn.softmax(tf_output, axis=1)
    label = tf.argmax(tf_prediction, axis=1)
    ans.append(label.numpy()[0])
    
df_out = pd.DataFrame({"id":data_df.id,'emotion': ans})
df_out
# df_out.to_csv("yeelongma.csv", index=False)

,id,emotion
0,8_1,4
1,8_2,3
2,8_3,4
3,8_4,4
4,8_5,5
...,...,...
4929,4188_2,6
4930,4188_3,3
4931,4188_4,6
4932,4188_5,3


In [14]:
ans = le.inverse_transform(ans)
df_out = pd.DataFrame({"id":data_df.id,'emotion': ans})
df_out.to_csv("yeelongma.csv", index=False)